Step 0: Importing Relevant Packages (0%)

In [32]:
# Web Scraping Libraries
from bs4 import BeautifulSoup  # for scraping
import requests  # for sending HTTP requests
# Data Processing Libraries
import csv
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Step 1: Crawl a Real World Data Set (2%)

In [3]:
url = "https://www.worldometers.info/coronavirus/#main_table"  # Web address of data
response = requests.get(url)  # Performing a get request for the data at the url location
soup_object = BeautifulSoup(response.content)  # Parsing the request response to create a soup_object that can be searched for a table
covid_table = soup_object.find("table")  # Searching the soup_object for a table

with open("Live_Covid_Data.csv", "w", newline="") as csv_file:  # Opening a CSV file to be written into
    writer = csv.writer(csv_file)  # Defining the writer as a csv writer that writes to the opened file

    for row in covid_table.find_all("tr"):  # Iterate through the rows of the table, "tr" points to a row in the table
        cells = row.find_all("td")  # Get the cells in the row, "td" points to a cell in the row

        if cells:  # If there are cells...
            values = [cell.text for cell in cells]  # Values is a list of values for each cell in the row of cells
            writer.writerow(values)  # Write the values to the CSV


Step 2: Data Preparation and Cleaning (5%)

In [41]:
# Creating list of column names
col_names = ["Country", "Total Cases", "New Cases", "Total Deaths", "New Deaths", "Total Recovered", "New Recovered", "Active Cases", "Critical Cases",
             "Total Cases / 1M People", "Total Deaths / 1M People", "Total Tests", "Total Tests / 1M People", "Population", "Continent", "One Case / n People",
             "One Death / n People", "One Test / n People", "New Cases / 1M People", "New Deaths / 1M People", "Active Cases / 1M People"]

# Reading the csv to a dataframe with the column names specified, encoding with "latin-1" to get around "utf-8" decoding issue.
df = pd.read_csv("Live_Covid_Data.csv", names=col_names, header=None, encoding="latin-1")

# Tidying up the names of the continent by removing the "\n"'s on it, i.e. "\nNorth America\n"
old_continent_list = df.iloc[0:7]["Country"].tolist()  # Locating the continent names and putting them into a list
new_continent_list = []  # Defining a list for the formatted continent names
for old_continent in old_continent_list:
     new_continent = "Antarctica" if old_continent == "\n\n" else old_continent.replace("\n", "")
     if new_continent == "Oceania":
         new_continent = "Australia/Oceania"
     new_continent_list.append(new_continent)

# Changing the index to the name of the Country/Continent
df.set_index("Country", inplace=True)

# Changing the index name to the re-formatted continents. set_value has been depricated, also before setting the index as Country the continent rows had
# an index of NaN so at[row, col] couldn't be used to assign values either. Therefore, application of the reformatted names had to after set_index was done.
n = 0
for new_continent in new_continent_list:
    df.rename(index={old_continent_list[n]: new_continent}, inplace=True)
    n += 1

# Moving country population to first column as it is important for comparison to total cases etc
country_pop_col = df.pop("Population")
df.insert(0, "Population", country_pop_col)

# Removing non-ccountries/territories from the DF
drop_list = ["Diamond Princess", "MS Zaandam", "Total:"]
for item in drop_list:
    df.drop(index=item, axis=0, inplace=True)


# Calculating the population of each continent and the world
global_population = 0
for continent in new_continent_list:
    population_list = df.loc[(df["Continent"] == continent) & (df.index != continent), "Population"].tolist()
    continent_population = 0
    for population in population_list:
        new_population = re.sub(",", "", population)
        continent_population += int(new_population)
    global_population += continent_population
    df.at[continent, "Population"] = continent_population
df.at["World", "Population"] = global_population

df

,Population,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Critical Cases,Total Cases / 1M People,Total Deaths / 1M People,Total Tests,Total Tests / 1M People,Continent,One Case / n People,One Death / n People,One Test / n People,New Cases / 1M People,New Deaths / 1M People,Active Cases / 1M People
Country,,,,,,,,,,,,,,,,,,,,
North America,598140916,"121,262,081","+5,968","1,578,160",28.0,"116,064,929","+5,895","3,618,992","9,463",NaN,NaN,NaN,NaN,North America,\n,NaN,NaN,NaN,NaN,NaN
Asia,4711356783,"206,595,856","+323,654","1,511,157",635.0,"193,360,313","+100,774","11,724,386","11,331",NaN,NaN,NaN,NaN,Asia,\n,NaN,NaN,NaN,NaN,NaN
Europe,747543837,"242,719,800","+15,247","1,980,119",110.0,"236,784,979","+66,776","3,954,702","7,569",NaN,NaN,NaN,NaN,Europe,\n,NaN,NaN,NaN,NaN,NaN
South America,437694443,"66,758,003","+3,145","1,341,406",1.0,"64,253,133","+23,085","1,163,464","10,341",NaN,NaN,NaN,NaN,South America,\n,NaN,NaN,NaN,NaN,NaN
Australia/Oceania,43469030,"13,582,848","+2,758","23,298",24.0,"13,232,703",+20,"326,847",161,NaN,NaN,NaN,NaN,Australia/Oceania,\n,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wallis and Futuna,"10,982",761,NaN,7,NaN,438,NaN,316,NaN,"69,295",637,"20,508","1,867,419",Australia/Oceania,14,"1,569",1.0,NaN,NaN,"28,774"
Niue,"1,622",458,+21,,NaN,290,+20,168,NaN,"282,367",NaN,NaN,NaN,Australia/Oceania,4,NaN,NaN,"12,947",NaN,"103,576"
Vatican City,799,29,NaN,,NaN,29,NaN,0,NaN,"36,295",NaN,NaN,NaN,Europe,28,NaN,NaN,NaN,NaN,NaN


Step 3: Exploratory Data Analysis (8%)

Step 4a: Question 1 (10%)

Step 4b: Question 2 (10%)

Step 4c: Question 3 (10%)

Step 5: Summary and Conclusion (5%)

Write summary/conc in here